<a href="https://colab.research.google.com/github/Pedrest15/Python/blob/main/OtimizacaoMisturas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Descrição do Problema

**Problema de correção de mistura**

Joãozinho trabalha na fabricação de remédio para gripe. Mais especificamente, ele trabalha na parte de inspeção: antes da mistura ser transformada em comprimidos, Joãozinho precisa corrigir, se necessário, as quantidades de cada componente do remédio.

Supondo que o remédio tenha 3 componentes: dipirona, maleato de clorfeniramina e cafeína. Joãozinho acaba de receber uma quantidade de 1kg de mistura e consta que nela há 30% de dipirona, 10% de maleato de clorfeniramina e 20% de cafeína. Entretanto, as as porcentagens deveriam ser de 33% a 47% de dipirona, 15% a 28% de maleato de clorfeniramina e 11% a 50% de cafeína.

Normalmente, Joãozinho adicionaria os componentes puros e solução neutra (matéria para reduzir a concentração dos componentes) até balancear a mistura, mas por um descuído da empresa se esgotaram os estoques deles. Para piorar, a empresa precisa entregar os remédios até o fim do dia, caso contrário será multada.

Porém, como Joãozinho é experiente nesse ramo, ele teve a seguinte ideia: além da mistura que acabou de receber, ele também possuía 3 outras misturas defeituosas cujas composições e quantidades disponíveis são:

* Mistura 1: 50% de dipirona, 40% de maleato de clorfeniramina e 10% de cafeína. Quantia = 1,7 kg.
* Mistura 2: 10% de dipirona, 40% de maleato de clorfeniramina e 50% de cafeína. Quantia = 1,5 kg.
* Mistura 3: 70% de dipirona, 20% de maleato de clorfeniramina e 10% de cafeína. Quantia = 1,3 kg.

Então ele pensou em adicionar essas outras misturas na que acabou de receber para balancear os componentes. Seu objetivo é obter a maior quantidade de mistura com a porcentagem correta dos componentes. Crie um modelo de otimização LINEAR que resolva esse problema (ou seja, maximize o uso das misturas).

OBS: Na função objetivo do modelo, não adicione a quantia inicial de mistura.



### Resolução

In [1]:
#importando biblioteca do modelo
%pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB

Looking in indexes: https://pypi.gurobi.com, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.6 MB/s eta 0:00:00


In [4]:
#criando modelo
model = gp.Model("Misturas")

#instanciando as variaveis
x1 = model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1.7, name="Mistura1")
x2 = model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1.5, name="Mistura2")
x3 = model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1.3, name="Mistura3")

#definindo a funcao objetivo
model.setObjective(x1 + x2 + x3, GRB.MAXIMIZE)

#definindo as restricoes do problema
model.addConstr(0.5*x1 + 0.1*x2 + 0.7*x3 + 0.3 <= 0.47*(x1+x2+x3+1), name="Max_dipirona")

model.addConstr(0.5*x1 + 0.1*x2 + 0.7*x3 + 0.3 >= 0.33*(x1+x2+x3+1), name="Min_dipirona")

model.addConstr(0.4*x1 + 0.4*x2 + 0.2*x3 + 0.1 <= 0.28*(x1+x2+x3+1), name="Max_cloro")

model.addConstr(0.4*x1 + 0.4*x2 + 0.2*x3 + 0.1 >= 0.15*(x1+x2+x3+1), name="Min_cloro")

model.addConstr(0.1*x1 + 0.5*x2 + 0.1*x3 + 0.2 <= 0.5*(x1+x2+x3+1), name="Max_cafeina")

model.addConstr(0.1*x1 + 0.5*x2 + 0.1*x3 + 0.2 >= 0.11*(x1+x2+x3+1), name="Min_cafeina")

model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 3 columns and 17 nonzeros
Model fingerprint: 0x20ef0b90
Coefficient statistics:
  Matrix range     [1e-02, 4e-01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [3e-02, 3e-01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 3 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.6666667e+00   1.066667e+00   0.000000e+00      0s
       1    3.6666667e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.666666667e+00


In [6]:
print("Função Objetivo: "+str(model.objVal))

print("Qnt da Mistura 1: "+str(x1.X))
print("Qnt da Mistura 2: "+str(x2.X))
print("Qnt da Mistura 3: "+str(x3.X))

Função Objetivo: 3.666666666666667
Qnt da Mistura 1: 1.7
Qnt da Mistura 2: 0.6666666666666672
Qnt da Mistura 3: 1.3


Considerando agora uma proporção de no mínimo 3/2 entre as misturas 1 e 2, e 1/2 entre as misturas 1 e 3, reformule o modelo.

In [16]:
#criando modelo
model2 = gp.Model("Misturas2")

x1 = model2.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1.7, name="Mistura1")
x2 = model2.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1.5, name="Mistura2")
x3 = model2.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1.3, name="Mistura3")

model2.setObjective(x1 + x2 + x3, GRB.MAXIMIZE)

model2.addConstr(0.5*x1 + 0.1*x2 + 0.7*x3 + 0.3 <= 0.47*(x1+x2+x3+1), name="Max_dipirona")

model2.addConstr(0.5*x1 + 0.1*x2 + 0.7*x3 + 0.3 >= 0.33*(x1+x2+x3+1), name="Min_dipirona")

model2.addConstr(0.4*x1 + 0.4*x2 + 0.2*x3 + 0.1 <= 0.28*(x1+x2+x3+1), name="Max_cloro")

model2.addConstr(0.4*x1 + 0.4*x2 + 0.2*x3 + 0.1 >= 0.15*(x1+x2+x3+1), name="Min_cloro")

model2.addConstr(0.1*x1 + 0.5*x2 + 0.1*x3 + 0.2 <= 0.5*(x1+x2+x3+1), name="Max_cafeina")

model2.addConstr(0.1*x1 + 0.5*x2 + 0.1*x3 + 0.2 >= 0.11*(x1+x2+x3+1), name="Min_cafeina")

model2.addConstr(x2 >= (2/3) * x1, name="qnt21")
model2.addConstr(x3 >= (2) * x1, name="qnt31")

model2.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 3 columns and 21 nonzeros
Model fingerprint: 0xd6c22e38
Coefficient statistics:
  Matrix range     [1e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [3e-02, 3e-01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 6 rows, 3 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4500000e+00   0.000000e+00   0.000000e+00      0s
       0    3.4500000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.450000000e+00


In [18]:
print("Função Objetivo: "+str(model2.objVal))

print("Qnt da Mistura 1: "+str(x1.X))
print("Qnt da Mistura 2: "+str(x2.X))
print("Qnt da Mistura 3: "+str(x3.X))

Função Objetivo: 3.45
Qnt da Mistura 1: 0.65
Qnt da Mistura 2: 1.5
Qnt da Mistura 3: 1.3
